In [2]:
#! pip install keras

     |████████████████████████████████| 1.3 MB 9.5 MB/s            


In [6]:
#! pip install tensorflow

In [1]:

from collections import Counter
from keras.preprocessing.text import Tokenizer
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import sys
import os
import json 
import random
import glob
import pandas as pd
import pickle
import numpy as np
from pprint import pprint,pformat
from metricUtils import tn, fp, tp, fn, precision, recall, fpr, tpr, tnr, f1, auc, mcc
import seaborn as sn
import matplotlib.pyplot as plt

In [22]:
# Merge all Json files in one Json file
def merge_json_files(path):
    result = []
    for f in glob.glob(path+"/*.json"):
        with open(f, "r") as infile:
            result.append(json.load(infile))
            
#     print(result)
    with open(path+"/merged_test_cases_file.json", "w") as outfile:
         json.dump(result, outfile)

In [24]:
path='../data/me'
merge_json_files(path)

In [9]:
# with open("../data/me/merged_test_cases_file.json", "r") as fp:
#     print(pformat(json.load(fp)))

In [2]:
# Read dataset.json

path='../data/me/merged_test_cases_file.json'
with open(path, 'r') as json_file:
    data = json.load(json_file)


# Options
percentage =20
numberOfFlakyToCreate = int( len(data) * percentage / 100 )
 # convert label [False.True] ==> [0,1]


# Creating list of random integers
randomlist = []
while len(randomlist) != numberOfFlakyToCreate:
    n = random.randint(0, len(data)-1)
    if n not in randomlist:
        randomlist.append(n)

# We put all Label to 0
for test in data:
#     print(test['Label'])
    test["Label"] = False


# We randomly affect test to Label 1
for i in range(0, len(data)):
    if i in randomlist:
        data[i]["Label"] = True


# Check up
c = 0
for test in data:
    if test["Label"] == True:
        c += 1

      

print("Data length: ", len(data))
print("Percentage of Flaky Tests wished: ", percentage)
print("Number of Flaky Tests to create: ", numberOfFlakyToCreate)
print(c, "Flaky Tests created.")

saveResults(data, "newexperiment")




Data length:  24
Percentage of Flaky Tests wished:  20
Number of Flaky Tests to create:  4
4 Flaky Tests created.


NameError: name 'saveResults' is not defined

In [10]:
def saveResults(dic, name):
    """Save results to file"""
    fileName = "../data/me/" + name + ".json"
    with open(fileName, 'w') as json_file:
        json.dump(dic, json_file, indent=4) 



In [34]:
data=pd.read_csv('../data/oracle/oracle.csv')

In [35]:
data.columns

Index(['Unnamed: 0', 'repo_name', 'file_name', 'class_name', 'func_name',
       'AssertionRoulette', 'ConditionalTestLogic',
       'ConstructorInitialization', 'DefaultTest', 'DuplicateAssertion',
       'EmptyTest', 'ExceptionHandling', 'GeneralFixture', 'IgnoredTest',
       'LackCohesion', 'MagicNumberTest', 'ObscureInLineSetup',
       'RedundantAssertion', 'RedundantPrint', 'SleepyTest',
       'SuboptimalAssert', 'TestMaverick', 'UnknownTest', 'body', 'voc_size',
       'important_voc', 'flaky'],
      dtype='object')

In [36]:
data['ConstructorInitialization'].dtype

dtype('bool')

In [37]:
test_small=['AssertionRoulette', 'ConditionalTestLogic',
       'ConstructorInitialization', 'DefaultTest', 'DuplicateAssertion',
       'EmptyTest', 'ExceptionHandling', 'GeneralFixture', 'IgnoredTest',
       'LackCohesion', 'MagicNumberTest', 'ObscureInLineSetup',
       'RedundantAssertion', 'RedundantPrint', 'SleepyTest',
       'SuboptimalAssert', 'TestMaverick', 'UnknownTest']
data_test_smell=data[test_small]
data_test_smell

,AssertionRoulette,ConditionalTestLogic,ConstructorInitialization,DefaultTest,DuplicateAssertion,EmptyTest,ExceptionHandling,GeneralFixture,IgnoredTest,LackCohesion,MagicNumberTest,ObscureInLineSetup,RedundantAssertion,RedundantPrint,SleepyTest,SuboptimalAssert,TestMaverick,UnknownTest
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5714,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False
5715,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5716,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True
5717,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False


In [40]:
# data = shuffle(data)
body = data['body'].values
#                 print(body[0])

# Building Tokenizer and Vocabulary
tokenizer = Tokenizer(lower=False, num_words=100, filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(body)
print("Vocabulary size: ", len(tokenizer.word_index) + 1)
print("Most important words:")
print(len(list(tokenizer.word_index.keys())[:10000]), "\n")
TK = tokenizer.texts_to_matrix(body, mode='count')
type(TK)

Vocabulary size:  8967
Most important words:
8966 



numpy.ndarray

In [42]:
TK_df = pd.DataFrame(data=TK)
TK_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,4.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,5.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,13.0,0.0,1.0,0.0,2.0,1.0,12.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5714,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5715,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5716,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5717,0.0,17.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:

X=pd.concat([data_test_smell, TK_df], axis=1)
X

,AssertionRoulette,ConditionalTestLogic,ConstructorInitialization,DefaultTest,DuplicateAssertion,EmptyTest,ExceptionHandling,GeneralFixture,IgnoredTest,LackCohesion,...,90,91,92,93,94,95,96,97,98,99
0,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,True,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,True,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,True,True,False,False,False,False,False,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5714,False,False,False,False,False,False,False,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5715,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5716,False,False,False,False,False,False,False,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5717,True,False,False,False,False,False,False,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
y = data['flaky'].values
y.shape

(5719,)

In [48]:
# Info
# Metric inspired by https://towardsdatascience.com/metrics-for-imbalanced-classification-41c71549bbb5
# Change parameters below (K and nbTree)

finalScores = []

def main():


    # Parameters
    k = 10
    nbTrees = 100
    modes = ["binary", "count", "tfidf", "freq"]  # ["tfidf"] #
    numWords =[100, 2000]
    lowerStates =[True, False]  # [True] # 
    

    # Load Data
    data=pd.read_csv('../data/oracle/oracle.csv')
    
    
#     print("Data length: ", len(data))
  
    for i in tqdm(range(0, len(modes))):
        mode = modes[i]
        for numWord in numWords:
            for lowerState in lowerStates:
               
                # Shuffle Data
                data = shuffle(data)
                #extract test smells
                data_test_smell=data[test_small]
              
                # Building Tokenizer and Vocabulary
                body = data['body'].values
                tokenizer = Tokenizer(lower=lowerState, num_words=numWord, filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
                tokenizer.fit_on_texts(body)
#                 print("Vocabulary size: ", len(tokenizer.word_index) + 1)
#                 print("Most important words:")
#                 print(len(list(tokenizer.word_index.keys())[:10000]), "\n")
                TK = tokenizer.texts_to_matrix(body, mode=mode)
                TK_df = pd.DataFrame(data=TK)
                X=pd.concat([data_test_smell, TK_df], axis=1)
#                 X=TK
#                 X=data_test_smell
                y = data['flaky'].values

                # Random Forest Model
                classifierKFold = RandomForestClassifier(n_estimators = nbTrees, random_state = 0) 
                               
            
                
                # Cross validation, K = 10, using stratified folds
                scoring = {
                    'precision': make_scorer(precision), 
                    'recall': make_scorer(recall), 
                    'f1': make_scorer(f1), 
                    'auc': make_scorer(auc), 
                    'fpr': make_scorer(fpr), 
                    'tnr': make_scorer(tnr), 
                    'mcc': make_scorer(mcc)
                }

                scores = cross_validate(classifierKFold, X, y, cv=k, scoring=scoring, verbose=0, n_jobs=8)

                # Display results
                # print("\nMetrics")
                # displayScores(scores['test_precision'], "Precision")
                # displayScores(scores['test_recall'], "Recall")
                # displayScores(scores['test_f1'], "F1")
                # displayScores(scores['test_auc'], "AUC")
                # displayScores(scores['test_mcc'], "MCC")

                # Save score
                o = {
                    "variables": {
                        "mode": mode,
                        "numWord": numWord,
                        "lowerState": lowerState
                        
                    },
                    "Precision": round(np.nanmean(scores['test_precision']), 2),
                    "Recall": round(np.nanmean(scores['test_recall']), 2),
                    "F1": round(np.nanmean(scores['test_f1']), 2),
                    "AUC": round(np.nanmean(scores['test_auc']), 2),
                    "MCC": round(np.nanmean(scores['test_mcc']), 2)
                }
                finalScores.append(o)

    # Display final scores
    sortedScores = sorted(finalScores, key=lambda x: x["F1"], reverse=True)

    # Best / Worst config
    print("\nBest configuration:")
    pprint(sortedScores[:1])
    print("\nWorst configuration:")
    pprint(sortedScores[-1:])

    # All config
    # pprint(sortedScores)

def displayScores(scores, title):
    print("\n",title,":", sep="")
    #print("Scores: ", scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (np.nanmean(scores), np.nanstd(scores) * 2))




In [49]:
main()

100%|██████████| 4/4 [02:18<00:00, 34.53s/it]


Best configuration:
[{'AUC': 0.92,
  'F1': 0.91,
  'MCC': 0.91,
  'Precision': 0.98,
  'Recall': 0.85,
  'variables': {'lowerState': True, 'mode': 'binary', 'numWord': 2000}}]

Worst configuration:
[{'AUC': 0.9,
  'F1': 0.87,
  'MCC': 0.87,
  'Precision': 0.97,
  'Recall': 0.8,
  'variables': {'lowerState': True, 'mode': 'binary', 'numWord': 100}}]


# Just Voc [100,2000]

Best configuration:
[{'AUC': 0.92,
  'F1': 0.91,
  'MCC': 0.91,
  'Precision': 0.98,
  'Recall': 0.85,
  'variables': {'lowerState': True, 'mode': 'binary', 'numWord': 2000}}]

Worst configuration:
[{'AUC': 0.9,
  'F1': 0.88,
  'MCC': 0.88,
  'Precision': 0.98,
  'Recall': 0.81,
  'variables': {'lowerState': True, 'mode': 'freq', 'numWord': 2000}}]

# VOC+smells [100,2000]
Best configuration:
[{'AUC': 0.93,
  'F1': 0.91,
  'MCC': 0.91,
  'Precision': 0.98,
  'Recall': 0.86,
  'variables': {'lowerState': True, 'mode': 'binary', 'numWord': 2000}}]

Worst configuration:
[{'AUC': 0.9,
  'F1': 0.88,
  'MCC': 0.88,
  'Precision': 0.98,
  'Recall': 0.81,
  'variables': {'lowerState': True, 'mode': 'freq', 'numWord': 2000}}]
  
# 100. depth
Best configuration:
[{'AUC': 0.93,
  'F1': 0.91,
  'MCC': 0.91,
  'Precision': 0.99,
  'Recall': 0.86,
  'variables': {'lowerState': True, 'mode': 'count', 'numWord': 2000}}]

Worst configuration:
[{'AUC': 0.9,
  'F1': 0.87,
  'MCC': 0.88,
  'Precision': 0.97,
  'Recall': 0.8,
  'variables': {'lowerState': False, 'mode': 'freq', 'numWord': 100}}]

In [97]:
x=data[data['flaky']==True]
x.shape

(236, 27)

# EXPEREMENT 2

In [30]:
test_small=[ 'ConditionalTestLogic',
       'ConstructorInitialization', 'DefaultTest', 'DuplicateAssertion',
       'EmptyTest', 'ExceptionHandling', 'GeneralFixture', 'IgnoredTest',
       'LackCohesion', 'MagicNumberTest', 'ObscureInLineSetup',
       'RedundantAssertion', 'RedundantPrint', 'SleepyTest',
       'SuboptimalAssert', 'TestMaverick', 'UnknownTest']
data_test_smell=data[test_small]
data_test_smell

NameError: name 'data' is not defined

In [31]:
# Info
# Metric inspired by https://towardsdatascience.com/metrics-for-imbalanced-classification-41c71549bbb5
# Change parameters below (K and nbTree)

finalScores = []

def main():


    # Parameters
    k = 10
    nbTrees = 100
    modes = ["binary", "count", "tfidf", "freq"]  # ["tfidf"] #
    numWords =[100, 2000]
    lowerStates =[True, False]  # [True] # 
    

    # Load Data
    data=pd.read_csv('../data/oracle/oracle.csv')
    data.drop('AssertionRoulette', axis=1, inplace=True)
  
#     corrMatrix = data.corr().abs()
# #     s = corrMatrix.unstack()
# #     so = s.sort_values(kind="quicksort")
# #     print (so)
    
#     fig, ax = plt.subplots(figsize=(15,15))         # Sample figsize in inches
#     sn.heatmap(corrMatrix, annot=True)
#     plt.show()
#     print("Data length: ", len(data))
  
    for i in tqdm(range(0, len(modes))):
        mode = modes[i]
        for numWord in numWords:
            for lowerState in lowerStates:
               
                # Shuffle Data
#                 data = shuffle(data)
                #extract test smells
                data_test_smell=data[test_small]
              
                # Building Tokenizer and Vocabulary
                body = data['body'].values
                tokenizer = Tokenizer(lower=lowerState, num_words=numWord, filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
                tokenizer.fit_on_texts(body)
#                 print("Vocabulary size: ", len(tokenizer.word_index) + 1)
#                 print("Most important words:")
#                 print(len(list(tokenizer.word_index.keys())[:10000]), "\n")
                TK = tokenizer.texts_to_matrix(body, mode=mode)
                TK_df = pd.DataFrame(data=TK)
                X=pd.concat([data_test_smell, TK_df], axis=1)
#                 X=TK
#                 X=data_test_smell
                y = data['flaky'].values

                # Random Forest Model
                classifierKFold = RandomForestClassifier(n_estimators = nbTrees, random_state = 0) 
                               
            
                
                # Cross validation, K = 10, using stratified folds
                scoring = {
                    'precision': make_scorer(precision), 
                    'recall': make_scorer(recall), 
                    'f1': make_scorer(f1), 
                    'auc': make_scorer(auc), 
                    'fpr': make_scorer(fpr), 
                    'tnr': make_scorer(tnr), 
                    'mcc': make_scorer(mcc)
                }

                scores = cross_validate(classifierKFold, X, y, cv=k, scoring=scoring, verbose=0, n_jobs=10)

                # Display results
                # print("\nMetrics")
                # displayScores(scores['test_precision'], "Precision")
                # displayScores(scores['test_recall'], "Recall")
                # displayScores(scores['test_f1'], "F1")
                # displayScores(scores['test_auc'], "AUC")
                # displayScores(scores['test_mcc'], "MCC")

                # Save score
                o = {
                    "variables": {
                        "mode": mode,
                        "numWord": numWord,
                        "lowerState": lowerState
                        
                    },
                    "Precision": round(np.nanmean(scores['test_precision']), 2),
                    "Recall": round(np.nanmean(scores['test_recall']), 2),
                    "F1": round(np.nanmean(scores['test_f1']), 2),
                    "AUC": round(np.nanmean(scores['test_auc']), 2),
                    "MCC": round(np.nanmean(scores['test_mcc']), 2)
                }
                finalScores.append(o)

    # Display final scores
    sortedScores = sorted(finalScores, key=lambda x: x["F1"], reverse=True)

    # Best / Worst config
    print("\nBest configuration:")
    pprint(sortedScores[:1])
    print("\nWorst configuration:")
    pprint(sortedScores[-1:])

    # All config
    # pprint(sortedScores)

def displayScores(scores, title):
    print("\n",title,":", sep="")
    #print("Scores: ", scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (np.nanmean(scores), np.nanstd(scores) * 2))





In [32]:
main()

100%|██████████| 4/4 [01:20<00:00, 20.23s/it]


Best configuration:
[{'AUC': 0.92,
  'F1': 0.91,
  'MCC': 0.91,
  'Precision': 0.98,
  'Recall': 0.85,
  'variables': {'lowerState': True, 'mode': 'count', 'numWord': 2000}}]

Worst configuration:
[{'AUC': 0.89,
  'F1': 0.87,
  'MCC': 0.87,
  'Precision': 0.97,
  'Recall': 0.78,
  'variables': {'lowerState': True, 'mode': 'freq', 'numWord': 100}}]
